In [1]:
# Import libraries
from functions import *
from functions.bip_0340_reference import *

import json
import os
import subprocess
import time

# P2TR Key Path and Script Path

- The following material adapted from the Bitcoin Optech [Schnorr Taproot workshop](https://bitcoinops.org/en/schorr-taproot-workshop/).


# TapTree

In this chapter we consider how to commit multiple tapscripts to a taptweak. This can be achieved with a binary tree commitment structure. We will also introduce taproot descriptors, which are composed of tapscript descriptors and reflect the binary tree commitment structure of a taproot output.

* **Part 1 - Constructing a taptree**
    * Taptree commitments
    * Taproot descriptors
    * Taptree construction

In part 2, we consider spending the taproot output along the script path for taproot outputs with taptree commitments, which have more than 1 commited tapscript. This requires an inclusion proof for the tapscript being spent.

* **Part 2 - Taproot script path**
    * Script path spending for taptrees

## Part 1: Constructing a taptree

### Taptree binary tree commitments

Committing multiple tapscripts requires a commitment structure resembling merkle tree construction.

**The TapTree is different than the header merkle tree in the following ways:**

* Tapleaves can be located at different heights.
* Ordering of TapLeaves is determined lexicograpically.
* Location of nodes are tagged (No ambiguity of node type).
 
Internal nodes are called tapbranches, and are also computed with the `tagged_hash("Tag", input_data)` function.
 
Tagged hashes are particularly useful when building a taptree commitment. They prevent node height ambiguity currently found in the transaction merkle tree, which allows an attacker to create a node which can be reinterpreted as either a leaf or internal node. Tagged hashes ensure that a tapleaf cannot be misinterpreted as an internal node and vice versa.

![test](images/taproot-workshop/taptree0.jpg)

#### _Programming Exercise 2.4.1:_ Compute a taptweak from a taptree

In the cell below, we will commit three pay-to-pubkey scripts to a taptweak and then derive the bech32m address. We will use the same merkle tree structure as in the previous illustration.

1. Compute TapLeaves A, B and C.
2. Compute Internal TapBranch AB.
3. Compute TapTweak
4. Derive the bech32m address.

In [2]:
TAPSCRIPT_VER = bytes([0xc0])  # See tapscript chapter for more details.
internal_privkey = bytes.fromhex("83a5f1039118fbb4276cac2db41d236c1c1790d97d955c228fa3bde439fbec2a")
internal_pubkey = tr.pubkey_gen(internal_privkey)


# Derive pay-to-pubkey scripts
privkeyA = bytes.fromhex("1059bf26660804ced9a3286a16497d7e70692d14dc04e1220c2dbef3667b74f7")
pubkeyA = tr.pubkey_gen(privkeyA)
privkeyB = bytes.fromhex("2b22bf11ab862a35f16301c0afc7afe60f66d31fc29645f79c2ab43655e65d33")
pubkeyB = tr.pubkey_gen(privkeyB)
privkeyC = bytes.fromhex("7f8b28e51da049bf63e31d3a3261579c0f5c1fc8058c65a79482814e5061f9f6")
pubkeyC = tr.pubkey_gen(privkeyC)

# Pay to pubkey scripts
scriptA = b"\x20" + pubkeyA + b"\xac"
scriptB = b"\x20" + pubkeyB + b"\xac"
scriptC = b"\x20" + pubkeyC + b"\xac"

# Method: Returns tapbranch hash. Child hashes are lexographically sorted and then concatenated.
# l: tagged hash of left child
# r: tagged hash of right child
def tapbranch_hash(l, r):
    return tagged_hash("TapBranch", b''.join(sorted([l,r])))

# 1) Compute TapLeaves A, B and C.
# Method: pushbytes(data) is a function which adds compactsize to input data.
hash_inputA =  TAPSCRIPT_VER + pushbytes(scriptA)
hash_inputB =  TAPSCRIPT_VER + pushbytes(scriptB)
hash_inputC =  TAPSCRIPT_VER + pushbytes(scriptC)
taggedhash_leafA =  tagged_hash("TapLeaf", hash_inputA)
taggedhash_leafB =  tagged_hash("TapLeaf", hash_inputB)
taggedhash_leafC =  tagged_hash("TapLeaf", hash_inputC)

# 2) Compute Internal node TapBranch AB.
# Method: use tapbranch_hash() function
internal_nodeAB = tapbranch_hash(taggedhash_leafA, taggedhash_leafB)

# 3) Compute TapTweak.
rootABC =  tapbranch_hash(internal_nodeAB, taggedhash_leafC)
taptweak =  tagged_hash("TapTweak", internal_pubkey + rootABC)

# 4) Derive the bech32m address.
negated, taproot_pubkey = taproot_tweak_pubkey(internal_pubkey, rootABC)
print("TapTweak:", taptweak.hex())
spk = bytes.fromhex("5120") + taproot_pubkey
bech32m_address = spk_to_bech32(spk, 'regtest')
print('Bech32m address:', bech32m_address)

TapTweak: 8e53fc0da2e9e8e27404703010ae519e85576ef6c73dde1b2bffac7f17b114a9
Bech32m address: bcrt1p5jhcyymfj7tkgv0jca4pz7tx9uzvznu0mlfymey6ph63f9h8x0gs7683vc


Now we are ready to set up bitcoind and fund this address. 

In [3]:
# Setup bitcoind and fund the address
setup_regtest_bitcoind()
txid_to_spend, index_to_spend = fund_address(bech32m_address, 2.001)
print(f"UTXO: {txid_to_spend}, {index_to_spend}")

UTXO: 2c6dfbf42de43a5e8761755f8f7109a42cb55c2020773165e8d6fbba0751418e, 0


In [4]:
# Decode the receiver's address
receiver_address = 'bcrt1ql3e9pgs3mmwuwrh95fecme0s0qtn2880hlwwpw'
hrp = 'bcrt'
witver, witprog = bech32.decode(hrp, receiver_address)
pubkey_hash = bytearray(witprog)
receiver_spk = bytes.fromhex("0014") + pubkey_hash

# Create a new pubkey to use as a change output.
change_privkey = bytes.fromhex("2222222222222222222222222222222222222222222222222222222222222222")
change_pubkey = privkey_to_pubkey(change_privkey)

# Set our output amount (in satoshis) and scriptPubkeys
output1_value_sat = int(float("1.5") * 100000000)
output1_spk = receiver_spk
output2_value_sat = int(float("0.5") * 100000000)
output2_spk = bytes.fromhex("0014") + hash160(change_pubkey)

## Part 2: Spending along the Key Path

In [5]:
taproot_privkey = taproot_tweak_seckey(internal_privkey, rootABC)

In [6]:
# VERSION
# version '2' indicates that we may use relative timelocks (BIP68)
version = bytes.fromhex("0200 0000")

# MARKER (new to segwit)
marker = bytes.fromhex("00")

# FLAG (new to segwit)
flag = bytes.fromhex("01")

# INPUTS
# We have just 1 input
input_count = bytes.fromhex("01")

# Convert txid and index to bytes (little endian)
txid = (bytes.fromhex(txid_to_spend))[::-1]
index = index_to_spend.to_bytes(4, byteorder="little", signed=False)

# For the unsigned transaction we use an empty scriptSig
scriptsig = bytes.fromhex("")

# use 0xffffffff unless you are using OP_CHECKSEQUENCEVERIFY, locktime, or rbf
sequence = bytes.fromhex("ffff ffff")

inputs = (
    txid
    + index
    + varint_len(scriptsig)
    + scriptsig
    + sequence
)

# OUTPUTS
# 0x02 for out two outputs
output_count = bytes.fromhex("02")

# OUTPUT 1 
output1_value = output1_value_sat.to_bytes(8, byteorder="little", signed=True)
# 'output1_spk' already defined at the start of the script

# OUTPUT 2
output2_value = output2_value_sat.to_bytes(8, byteorder="little", signed=True)
# 'output2_spk' already defined at the start of the script

outputs = (
    output1_value
    + varint_len(output1_spk)
    + output1_spk
    + output2_value
    + varint_len(output2_spk)
    + output2_spk
)

# LOCKTIME
locktime = bytes.fromhex("0000 0000")

unsigned_tx = (
    version
    + input_count
    + inputs
    + output_count
    + outputs
    + locktime
)
print("unsigned_tx: ", unsigned_tx.hex())

unsigned_tx:  02000000018e415107bafbd6e865317720205cb52ca409718f5f7561875e3ae42df4fb6d2c0000000000ffffffff0280d1f00800000000160014fc7250a211deddc70ee5a2738de5f07817351cef80f0fa0200000000160014531260aa2a199e228c537dfa42c82bea2c7c1f4d00000000


In [7]:
sighash_epoch = bytes.fromhex("00")
index_of_this_input = bytes.fromhex("0000 0000")

# Control
hash_type = bytes.fromhex("00") # SIGHASH_DEFAULT (a new sighash type meaning implied SIGHASH_ALL)

# Transaction data
sha_prevouts = sha256(txid + index)

input_amount_sat = int(2.001 * 100_000_000)
input_amounts = input_amount_sat.to_bytes(8, byteorder="little", signed=False)
sha_amounts = sha256(input_amounts)

sha_scriptpubkeys = sha256(
    varint_len(spk)
    + spk
)

sha_sequences = sha256(sequence)

sha_outputs = sha256(outputs) ######

# Data about this input
spend_type = bytes.fromhex("00") # no annex present

sig_msg = (
    sighash_epoch
    + hash_type
    + version
    + locktime
    + sha_prevouts
    + sha_amounts
    + sha_scriptpubkeys
    + sha_sequences
    + sha_outputs
    + spend_type
    + index_of_this_input
)

In [8]:
tag_hash = sha256("TapSighash".encode())
sighash = sha256(tag_hash + tag_hash + sig_msg)
print(sighash.hex())

67702a01986950b89016a977538096d49fe26c8bf81dd8ff5645c4cc8afd9653


In [9]:
aux_rand = bytes.fromhex("0000000000000000000000000000000000000000000000000000000000000000")

signature = tr.schnorr_sign(sighash, taproot_privkey, aux_rand)

# Only append the sighash type if it is not SIGHASH_DEFAULT (0x00)
if hash_type != b"\x00":
    signature += hash_type
    
signature.hex()

'bef4c867f41e5c72ad8d74c474a1b57b7dae35ca30350bc4d904de98c697f57d5866aa225db438f127d1ffb528c86637f11242800594f29960eebe5e211559e8'

In [10]:
witness = (
    bytes.fromhex("01") # one stack item in the witness
    + pushbytes(signature)
)
print(witness.hex())

# the final signed transaction
signed_tx = (
    version
    + marker
    + flag
    + input_count
    + inputs
    + output_count
    + outputs
    + witness
    + locktime
)

print("signed transaction: ",signed_tx.hex())

0140bef4c867f41e5c72ad8d74c474a1b57b7dae35ca30350bc4d904de98c697f57d5866aa225db438f127d1ffb528c86637f11242800594f29960eebe5e211559e8
signed transaction:  020000000001018e415107bafbd6e865317720205cb52ca409718f5f7561875e3ae42df4fb6d2c0000000000ffffffff0280d1f00800000000160014fc7250a211deddc70ee5a2738de5f07817351cef80f0fa0200000000160014531260aa2a199e228c537dfa42c82bea2c7c1f4d0140bef4c867f41e5c72ad8d74c474a1b57b7dae35ca30350bc4d904de98c697f57d5866aa225db438f127d1ffb528c86637f11242800594f29960eebe5e211559e800000000


In [11]:
new_tx_txid = subprocess.getoutput("bitcoin-cli -regtest testmempoolaccept " + "'[\"" +  signed_tx.hex() + "\"]'")
print(new_tx_txid)

[
  {
    "txid": "4790f79e3ee3548002d986438f4300d1393223fd5c994c961bcdb08f0d764e55",
    "wtxid": "6b181993a23069b6d3ea7e4b020fc63ba8c59cb0dff22da34cfd470120c5ee61",
    "allowed": true,
    "vsize": 130,
    "fees": {
      "base": 0.00100000
    }
  }
]


## Part 3: Spending along the Script Path

A Taproot output is spent along the script path with the following witness pattern:

* Witness to spend TapScript_A:

    * `[Stack element(s) satisfying TapScript_A]`
    * `[TapScript_A]` 
    * `[Controlblock c]`

Compared to the script spend path of a taproot with a single committed tapscript, the controlblock spending a taproot containing multiple tapscripts will also include a script inclusion proof.

* Controlblock c contains:

    * `[Tapscript Version]` 
        * `0xfe & c[0]`
    * `[Parity bit (oddness of Q's y-coordinate)]`
        * `0x01 & c[0]` 
    * `[Internal Public Key]` 
        * `c[1:33]`
    * `[Script Inclusion Proof]` 
        * `n x 32Bytes`
        
Note that this script inclusion proof is a 32B multiple and its size will depend on the position of tapscript in the taptree structure.

In [12]:
spend_type = bytes.fromhex("02")

In [13]:
sighash_epoch = bytes.fromhex("00")
index_of_this_input = bytes.fromhex("0000 0000")

# Control
hash_type = bytes.fromhex("00") # SIGHASH_DEFAULT (a new sighash type meaning implied SIGHASH_ALL)

# Transaction data
sha_prevouts = sha256(txid + index)

input_amount_sat = int(2.001 * 100_000_000)
input_amounts = input_amount_sat.to_bytes(8, byteorder="little", signed=False)
sha_amounts = sha256(input_amounts)

sha_scriptpubkeys = sha256(
    varint_len(spk)
    + spk
)

sha_sequences = sha256(sequence)

sha_outputs = sha256(outputs) ######

sig_msg = (
    sighash_epoch
    + hash_type
    + version
    + locktime
    + sha_prevouts
    + sha_amounts
    + sha_scriptpubkeys
    + sha_sequences
    + sha_outputs
    + spend_type # spend_type = 0x02 for script path
    + index_of_this_input
    # add script path information here
    + tagged_hash("TapLeaf", TAPSCRIPT_VER + pushbytes(scriptA))
    + bytes.fromhex("00") 
    + bytes.fromhex("ffffffff") # codeseparator_pos?
)

In [14]:
tag_hash = sha256("TapSighash".encode())
sighash = sha256(tag_hash + tag_hash + sig_msg)
print(sighash.hex())

514266e477b0854e1a50c6cfc23af3c242be69eba0886637ec4d476b7802bdf0


In [15]:
aux_rand = bytes.fromhex("0000000000000000000000000000000000000000000000000000000000000000")

signatureA = tr.schnorr_sign(sighash, privkeyA, aux_rand)

# Only append the sighash type if it is not SIGHASH_DEFAULT (0x00)
if hash_type != b"\x00":
    signature += hash_type
    
signature.hex()

'bef4c867f41e5c72ad8d74c474a1b57b7dae35ca30350bc4d904de98c697f57d5866aa225db438f127d1ffb528c86637f11242800594f29960eebe5e211559e8'

In [16]:
control_byte = (TAPSCRIPT_VER[0] | negated).to_bytes(1, "big")

control_block = (
    control_byte
    + internal_pubkey
    + taggedhash_leafB
    + taggedhash_leafC
    )

In [17]:
witness = (
    bytes.fromhex("03") 
    + varint_len(signatureA)
    + signatureA
    + varint_len(scriptA)
    + scriptA
    + varint_len(control_block)
    + control_block
)

# the final signed transaction
signed_script_path_tx = (
    version
    + marker
    + flag
    + input_count
    + inputs
    + output_count
    + outputs
    + witness
    + locktime
)

print("signed transaction: ",signed_script_path_tx.hex())

signed transaction:  020000000001018e415107bafbd6e865317720205cb52ca409718f5f7561875e3ae42df4fb6d2c0000000000ffffffff0280d1f00800000000160014fc7250a211deddc70ee5a2738de5f07817351cef80f0fa0200000000160014531260aa2a199e228c537dfa42c82bea2c7c1f4d0340229ccb3c23f371c76d937e4e54218d438e3e6847de3c49d35e3d64436a6e2ac4c8f4acc83b1a5e90e20953a3d9aebce04002d01164911190c3ff5fa04c0f0c312220731bbf2e7163d87b12d66d3795655790691e59802fd1c578c2e06287555e3c28ac61c1031845925dcca99bc5689ce422b9204ff9721de8416b984b8a6b930e30352225a1494f24cfdfc93532412675eece318c63414dd02b8750947177b141e082076e2b71b71e05a8cd9771485b9248025e46871caa6585164a1fdf9e634d61023fd800000000


In [18]:
new_tx_txid = subprocess.getoutput("bitcoin-cli -regtest testmempoolaccept " + "'[\"" +  signed_script_path_tx.hex() + "\"]'")
print(new_tx_txid)

[
  {
    "txid": "4790f79e3ee3548002d986438f4300d1393223fd5c994c961bcdb08f0d764e55",
    "wtxid": "01846ae8e95c3522a27c123a9fc65b0cf877cba4fc7951151ee77e5e10ee1013",
    "allowed": true,
    "vsize": 164,
    "fees": {
      "base": 0.00100000
    }
  }
]


## Quiz


 ## Answers
    

## Exercise
